# Last Away ~ all frequencies: BJH024

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns
import scipy
import joblib

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra



In [2]:
## Prep paths ##

subject = 'BJH024'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Neural Data

# load
last_away_ecpohs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(last_away_ecpohs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(last_away_ecpohs.get_annotations_per_epoch()) if  x]

# onlt good epochs
last_away_ecpohs = last_away_ecpohs[good_epochs]

# load behavioral data
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")

## Prep Behavioral Data
last_away_data['trial_numeric'] = last_away_data['trial_numeric'] - 1
last_away_data = last_away_data[last_away_data.index.isin(good_epochs)]

# set info as metadata
last_away_ecpohs.metadata = last_away_data

Reading /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_raw_clean_last_away_events.fif ...


/tmp/ipykernel_384103/870373520.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_raw_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_ecpohs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif")


    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/BJH024/ieeg/BJH024_bp_raw_clean_last_away_events-1.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
0 bad epochs dropped
Not setting metadata
220 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 5 columns


In [4]:
# separate ghost and no ghost trials

ghost_epochs = last_away_ecpohs["TrialType <= 16"]
no_ghost_epochs = last_away_ecpohs["TrialType > 16"]

In [7]:
## Dictionary of electrode locations ##
%debug
# Dictionary mapping ROI to elecs
ROIs = {
    'hc'     :['HL1', 'HL2', 'HL3', 'HL4', 'HL5',  'HR1', 'IL1', 'IL2', 'IL3', 'IL4', 'IR1', 'IR2', 'IR3', 'IR4', 'IR5'],
    'ofc'    :['AL1', 'AL2', 'AL3', 'AL4', 'AL5', 'AL6', 'AL7', 'AL8', 'AL9', 'AL10', 'AR1', 'AR2', 'AR3', 'AR4', 'AR5', 'AR6', 'AR7', 'AR8', 'AR9', 'BL1', 'BL2', 'BL3', 'BL4'],
    'amyg'   :['CR1', 'CR2', 'GL1', 'GL2', 'GL3', 'GL4', 'GL5', 'GR1'],
    'cing'   :['BR5', 'BR6', 'BR7', 'BR8', 'BR9', 'BR10'],
    'insula' :['CL4', 'CL5', 'CL6', 'CL7', 'CL8', 'CR5', 'CR6', 'CR7', 'CR8', 'GL7', 'GL8', 'GR3', 'GR4'],
    'dlpfc' :['CL13', 'CL14', 'CL15', 'CL16', 'CR12', 'CR13', 'CR14', 'CR15', 'CR16'],
    'ec' :[]
}

## prep lists

# primary ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in last_away_ecpohs.info['ch_names']]
bidx = len(last_away_ecpohs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    print(ix)
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

> /tmp/ipykernel_384103/95031559.py(49)<module>()
     47 for ix in range(0, len(pairs_name)):
     48     print(ix)
---> 49     if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
     50         hc_list.append(last_away_ecpohs.info['ch_names'][ix + bidx])
     51         hc_names.append(pairs_name[ix])



ipdb>  ix


0


ipdb>  pairs_name[ix][0]


'AL14'


...

ipdb>  pairs_name[ix]


['AL14']


ipdb>  pairs_name


[['AL14'], ['BL15'], ['BL16'], ['CL12'], ['CL13'], ['CL14'], ['CL15'], ['CL16'], ['DL11'], ['DL12'], ['DL13'], ['DL14'], ['DL15'], ['DL16'], ['IL11'], ['IL12'], ['IL13'], ['IL14'], ['AR14'], ['CR16'], ['DR11'], ['DR12'], ['DR13'], ['DR15'], ['DR16'], ['IR12'], ['IR13'], ['IR14'], ['JR13'], ['JR14'], ['STI'], ['AL1', 'AL2'], ['AL2', 'AL3'], ['AL3', 'AL5'], ['AL5', 'AL6'], ['AL6', 'AL7'], ['AL7', 'AL8'], ['AL8', 'AL9'], ['AL9', 'AL11'], ['AL11', 'AL12'], ['BL1', 'BL2'], ['BL2', 'BL3'], ['BL3', 'BL4'], ['BL4', 'BL5'], ['BL5', 'BL6'], ['BL6', 'BL7'], ['BL7', 'BL9'], ['BL9', 'BL10'], ['BL10', 'BL11'], ['BL11', 'BL12'], ['BL12', 'BL14'], ['CL1', 'CL2'], ['CL2', 'CL7'], ['CL7', 'CL8'], ['CL8', 'CL9'], ['CL9', 'CL10'], ['DL1', 'DL2'], ['DL2', 'DL3'], ['EL1', 'EL4'], ['EL4', 'EL5'], ['EL5', 'EL6'], ['EL6', 'EL7'], ['EL7', 'EL8'], ['EL8', 'EL9'], ['EL9', 'EL10'], ['EL10', 'EL11'], ['EL11', 'EL12'], ['FL1', 'FL2'], ['FL2', 'FL3'], ['FL3', 'FL4'], ['FL4', 'FL5'], ['FL5', 'FL6'], ['FL6', 'FL7'], ['

IndexError: list index out of range

In [ ]:
## funcions

def MM_compute_TFR(epochs, freqs, n_cycles, baseline, zscore=True, workers = 8):
    """ function to compute TFR via Morlet wavelets
    
    epochs:                     MNE epoch object with channels of interest
    freqs:                      list of frequencies, should be log spaced
    n_cycles:                   number of cycles, adjust with freqs to balance temporal and frequency resolution
    baseline:                   tuple that specifies the time in seconds that baseline should be calculated on 
                                (should be 1-2s smaller than epoch TFR was calculated on to exclude artificats on morlet waves
    zscore:                     boolean, if True zscores within trial and frequency band across the time window specified in baseline
    workers:                    number of threads to use while calculating TFR
    """
    print('computing TFR')
    TFR = mne.time_frequency.tfr_morlet(epochs,freqs,n_cycles,return_itc=False,
                                            average=False,  n_jobs = workers)
    print('log transforming')
    TFR.data = np.log(TFR.data)
    if zscore:
        print('z-scoring to baseline')
        bix = [a and b  for a, b in zip(TFR.times >= baseline[0], TFR.times <= baseline[1])]
        bmean = TFR.data[:,:,:,bix].mean(axis=(3),keepdims=True)
        bstd = TFR.data[:,:,:,bix].std(axis=(3),keepdims=True)
        TFR.data = (TFR.data - bmean) / bstd
    return TFR

def extract_freqs(lower_freq, higher_freq, freq_band, subdir, ROI, label, TFR, trials):
    """ function to extract and average the across the freqs within a given band and save out to csvs
    step is calculated based on getting ~4 samples per frequency cycle
    
    lower_freq, higher_freq:    non inclusive lower and upper bounds of the band
    freq_band:                  band name, as a string
    subdir:                     dir in sub/ieeg/ that specifies the time locking
    ROI:                        region name, as a string
    label:                      label, eg ghost, no ghost, choice locked etc, as a string
    TFR:                        MNE TFR object
    """
    
    # calculate step, ## note I made this up, but it seems reasonable?
    step = {
        'delta'      : int(np.floor(ghost_TFR.info['sfreq']/(2*4))),
        'theta'      : int(np.floor(ghost_TFR.info['sfreq']/(5*4))),
        'alpha'      : int(np.floor(ghost_TFR.info['sfreq']/(11*4))),
        'beta'       : int(np.floor(ghost_TFR.info['sfreq']/(22*4))),
        'gamma'      : int(np.floor(ghost_TFR.info['sfreq']/(50*4))),
        'hfa'        : int(np.floor(ghost_TFR.info['sfreq']/(110*4)))
    }
    
    # check if it needs to be calculated with subbands
    if freq_band == 'gamma' or freq_band == 'hfa':
        
        if ghost_TFR.info['sfreq'] < 1000:
                                raise Exception('Sampling Rate is below 1000, should calculate gamma or HFA') 
        
        subband_dict = {
            'gamma'    : [(30, 40), (35, 45), (40, 50), (45, 55), (50, 60), (55, 65), (60, 70)],
            'hfa'      : [(70, 90), (80, 100), (90, 110), (100, 120), (110, 130), (120, 140), (130, 150)]
        }
        
        for chix in range(len(TFR.ch_names)):
            subb_trial_power = []
            for subb in subband_dict[freq_band]:
                fidx = np.where((freqs > subb[0]) & (freqs < subb[1]))[0]
                subb_trial_power.append(TFR.data[:, chix, fidx, :].mean(axis=1))
            trial_power = np.mean(subb_trial_power, axis = 0)
            channel_df = pd.DataFrame(trial_power[:, ::step[freq_band]])
            channel_df["trial"] = trials
            channel_df.to_csv(f"{preproc_data_dir}/{subdir}/{TFR.ch_names[chix]}_{ROI}_trial_{freq_band}_{label}.csv")
    
    else:
        fidx = np.where((freqs > lower_freq) & (freqs < higher_freq))[0]
        for chix in range(len(TFR.ch_names)):
            trial_power = TFR.data[:, chix, fidx, :].mean(axis=1)
            channel_df = pd.DataFrame(trial_power[:, ::step[freq_band]])
            channel_df["trial"] = trials
            channel_df.to_csv(f"{preproc_data_dir}/{subdir}/{TFR.ch_names[chix]}_{ROI}_trial_{freq_band}_{label}.csv")

In [ ]:
# Set frequencies

freqs = np.logspace(start = np.log10(1), stop = np.log10(150), num = 80, base = 10, endpoint = True)
n_cycles = np.logspace(np.log10(2), np.log10(30), base = 10, num = 80)

# formulas to check bandwidth and time bin
band_width = (freqs / n_cycles) * 2
time_bin = n_cycles / freqs / np.pi
print(freqs)
print(n_cycles)
print(time_bin)
print(band_width)


# Main Regions of Interest

## Hippocampus

In [ ]:
hc_list

In [ ]:
# only ROI of interest
ghost_roi = ghost_epochs.copy().pick_channels(hc_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(hc_list)

In [ ]:
# Resample to 1000
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_roi.metadata.trial_numeric

In [ ]:
# compute TRF
ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
ghost_TFR.data.shape

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]

In [ ]:
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Hippocampal TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_hc_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Hippocampal TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_hc_noconflict.png', dpi=600)
fig.show()


## Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

## Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))

In [ ]:
# true difference matrix
true_diff_matrix = ghost_TFR.data[:, :, :, :].mean(axis = 0).mean(axis = 0) - no_ghost_TFR.data[:, :, :, :].mean(axis = 0).mean(axis = 0)


# initialize permuted matrix
permuted_matrix = np.nan * np.ones_like(true_diff_matrix)
permuted_matrices = np.repeat(permuted_matrix[np.newaxis,...], 1000, axis=0)

# get incex vectors
num_ghost_trials = ghost_TFR.data.shape[0]
num_no_ghost_trials = no_ghost_TFR.data.shape[0]
total_trials = num_ghost_trials + num_no_ghost_trials
trial_indices = np.arange(0, total_trials, 1)

# combine all tfrs
all_tfrs = np.concatenate([ghost_TFR.data[:, :, :, :] , no_ghost_TFR.data[:, :, :, :]], axis = 0)

for permute in range(0, 1000):
    
    trial_indices = np.arange(0, total_trials, 1)
    np.random.seed(permute)
    fake_ghost_indices = np.random.choice(a = trial_indices, size=num_ghost_trials, replace = False)
    fake_no_ghost_indicies = np.delete(trial_indices, fake_ghost_indices)


    fake_ghost_tfr = all_tfrs[fake_ghost_indices, :, :, :]
    fake_no_ghost_tfr = all_tfrs[fake_no_ghost_indicies, :, :, :]

    permuted_matrices[permute, :, :] = fake_ghost_tfr.mean(axis = 0).mean(axis = 0) - fake_no_ghost_tfr.mean(axis = 0).mean(axis = 0)


In [ ]:
# create dummy sig matrix
sig_matrix = np.full(true_diff_matrix.shape, False, dtype = 'bool')
pval_matrix = np.nan * np.ones_like(true_diff_matrix)


# find values at the extreme
for y in range(0,80):
    for x in range(0, 4001):

        permuted_dist = np.sort(permuted_matrices[:, y, x])

        if true_diff_matrix[y, x] <= permuted_dist[50] or true_diff_matrix[y, x] >=permuted_dist[950] :
            sig_matrix[y, x] = True
        else:
            sig_matrix[y, x] = False

sig_diff_matrix = np.where(sig_matrix, true_diff_matrix, np.nan)


In [ ]:

fig, ax = plt.subplots()
i1 = ax.imshow(true_diff_matrix, cmap = 'Greys', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
i2 = ax.imshow(sig_diff_matrix, cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i2)
ax.set_title("Hippocampus: Sig Difference Matrix")
fig.show()



# OFC

In [ ]:
ofc_list

In [ ]:
ghost_roi = ghost_epochs.copy().pick_channels(ofc_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(ofc_list)

In [ ]:
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_TFR = []
no_ghost_TFR = []

ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True, workers = 5)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True, workers = 5)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average OFC TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_ofc_conflict.png', dpi=600)
fig.show()

fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average OFC TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_ofc_noconflict.png', dpi=600)
fig.show()



### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))

# Amygdala

In [ ]:
ghost_roi = ghost_epochs.copy().pick_channels(amyg_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(amyg_list)

In [ ]:
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_TFR = []
no_ghost_TFR = []

ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Amygdala TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_amyg_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Amygdala TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_amyg_noconflict.png', dpi=600)
fig.show()

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))

## Cingulate

In [ ]:
cing_list

In [ ]:
ghost_roi = ghost_epochs.copy().pick_channels(cing_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(cing_list)

In [ ]:
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_TFR = []
no_ghost_TFR = []

ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Cingulate TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_cing_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Cingulate TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_cing_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [14, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))

# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
ghost_roi = ghost_epochs.copy().pick_channels(insula_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(insula_list)

In [ ]:
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_TFR = []
no_ghost_TFR = []

ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Insula TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_insula_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average Insula TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_insula_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
ghost_roi = ghost_epochs.copy().pick_channels(dlpfc_list)
no_ghost_roi = no_ghost_epochs.copy().pick_channels(dlpfc_list)

In [ ]:
ghost_roi= ghost_roi.resample(1000)
no_ghost_roi= no_ghost_roi.resample(1000)

In [ ]:
ghost_TFR = []
no_ghost_TFR = []

ghost_TFR = MM_compute_TFR(ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)
no_ghost_TFR = MM_compute_TFR(no_ghost_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)

In [ ]:
ghost_TFR.crop(tmin = -2, tmax = 2)
no_ghost_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]
fig, ax = plt.subplots()
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average dlPFC TFR in Conflict Trials")
fig.savefig(f'figures/{subject}_average_dlpfc_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots()
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin =-.2, vmax = .2)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title("Average dlPFC TFR in No-Conflict Trials")
fig.savefig(f'figures/{subject}_average_dlpfc_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots()
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots()
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -.4, vmax = .4)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_roi.metadata.trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_roi.metadata.trial_numeric.to_numpy(copy = True))